In [1]:
import pandas as pd

In [2]:
data_ = pd.read_csv('..\data\charts.csv')
data_.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [3]:
data = data_#.head(100000)

In [4]:
# Artist
df_artist = pd.DataFrame(data['artist'].unique(), columns=['name'])
df_artist = df_artist.assign(name=df_artist['name'].str.split(', ')).explode('name')
df_artist = pd.DataFrame(df_artist['name'].unique(), columns=['name'])
df_artist.reset_index(drop = True, inplace=True)
df_artist.insert(0, 'artist_id', range(1, 1 + len(df_artist)))

# Song
df_song = pd.DataFrame(data['title'].unique(), columns=['title'])
df_song.insert(0, 'song_id', range(1, 1 + len(df_song)))

# Trend
df_trend = pd.DataFrame(data['trend'].unique(), columns=['trend'])
df_trend.insert(0, 'trend_id', range(1, 1 + len(df_trend)))

# Day
df_day = pd.DataFrame(data['date'].unique(), columns=['date'])
df_day.insert(0, 'day_id', range(1, 1 + len(df_day)))

# Region
df_region = pd.DataFrame(data['region'].unique(), columns=['name'])
df_region.insert(0, 'region_id', range(1, 1 + len(df_region)))

# Category
df_category = pd.DataFrame(data['chart'].unique(), columns=['name'])
df_category.insert(0, 'category_id', range(1, 1 + len(df_category)))

In [5]:
artistDict = df_artist.set_index('name').T.to_dict('records')[0]
songDict = df_song.set_index('title').T.to_dict('records')[0]
trendDict = df_trend.set_index('trend').T.to_dict('records')[0]
dayDict = df_day.set_index('date').T.to_dict('records')[0]
regionDict = df_region.set_index('name').T.to_dict('records')[0]
categoryDict = df_category.set_index('name').T.to_dict('records')[0]

In [6]:
df = pd.DataFrame(columns = ['chart_id', 'position', 'song_id', 'day_id', 'region_id', 'category_id', 'trend_id', 'streams'])
df.to_csv('../data/df_chart.csv', mode='w', index = 'False')

In [7]:
start = 0 # enter the index of the next line to write (to continue enter (last chart_id)), for first time enter 0
end = 99999 # last index to write (last value of chart_id will be end+1)
data = data_.loc[start:end]
data.reset_index(drop = True, inplace=True)
print(f"Length of data: {len(data)}")

Length of data: 100000


In [8]:
saveVal = 10000 #how much record save in one loop
num = 10 #how many loops
for i in range(0, num):
    dataPart = data.loc[i*saveVal:(i+1)*saveVal-1].copy()
    # Chart
    df_chart = dataPart[['rank', 'title', 'date', 'region', 'chart', 'trend', 'streams']].drop_duplicates().reset_index().drop(columns = ['index'])
    df_chart = df_chart.rename(columns = {'title':'song_id', 'rank':'position', 'date':'day_id', 'region':'region_id', 'chart':'category_id', 'trend':'trend_id'})
    df_chart.insert(0, 'chart_id', range(start + i*saveVal + 1, start + i*saveVal + 1 + len(df_chart)))
    df_chart.replace({"song_id": songDict}, inplace=True) 
    df_chart.replace({"day_id": dayDict}, inplace=True) 
    df_chart.replace({"region_id": regionDict}, inplace=True) 
    df_chart.replace({"category_id": categoryDict}, inplace=True) 
    df_chart.replace({"trend_id": trendDict}, inplace=True)
    df_chart['streams'] = df_chart['streams'].astype(pd.Int64Dtype())

    df_chart.to_csv('../data/df_chart.csv', mode = 'a', header = False, index = False)
